In [12]:
# Import Libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from pickle import dump
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Train & Test Data

train_data = pd.read_csv("../data/diabetes_clean_train.csv")
test_data = pd.read_csv("../data/diabetes_clean_test.csv")

train_data.head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Age,Outcome
0,475,0,137,84,27,0,27.3,59,0
1,6,3,78,50,32,88,31.0,26,1
2,450,1,82,64,13,95,21.2,23,0
3,635,13,104,72,0,0,31.2,38,1
4,438,1,97,70,15,0,18.2,21,0


In [14]:
# Remove target variable from train data

X_train = train_data.drop(["Outcome", "Unnamed: 0"], axis = 1)
y_train = train_data["Outcome"]
X_test = test_data.drop(["Outcome", "Unnamed: 0"], axis = 1)
y_test = test_data["Outcome"]
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Age
0,0,137,84,27,0,27.3,59
1,3,78,50,32,88,31.0,26
2,1,82,64,13,95,21.2,23
3,13,104,72,0,0,31.2,38
4,1,97,70,15,0,18.2,21


In [15]:
# XGBoost Classifier

from xgboost import XGBClassifier

model = XGBClassifier(n_estimators = 2000, learning_rate = 0.001, random_state = 42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2000, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [16]:
# y-prediction

y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0])

In [17]:
# Report & Accuracy Score

report = classification_report(y_test, y_pred)
print(f'Regular Accuracy Score:\n {accuracy_score(y_test, y_pred)}')
print(report)

Regular Accuracy Score:
 0.8344827586206897
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       102
           1       0.71      0.74      0.73        43

    accuracy                           0.83       145
   macro avg       0.80      0.81      0.80       145
weighted avg       0.84      0.83      0.84       145



In [18]:
# Best Hyperparameters

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Create the XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2],
}

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(xgb_clf, param_distributions=param_dist, n_iter=50, scoring='accuracy', 
                                   cv=5, verbose=2, random_state=42, n_jobs=-1)

# Fit the model to find the best hyperparameters
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best Hyperparameters:", random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   0.1s


/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:34] WARNING: /w

[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=200, reg_alpha=0.1, reg_lambda=1.5, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=200, reg_alpha=0.1, reg_lambda=1.5, subsample=1.0; total time=   0.1s


/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:35] WARNING: /w

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=200, reg_alpha=0.1, reg_lambda=1.5, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=200, reg_alpha=0.1, reg_lambda=1.5, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.2, max_depth=7, min_child_weight=3, n_estimators=100, reg_alpha=0.1, reg_lambda=2, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.2, max_depth=7, min_child_weight=3, n_estimators=100, reg_alpha=0.1, reg_lambda=2, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=200, reg_alpha=0.1, reg_lambda=1.5, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.2, max_depth=7, min_child_weight=3, n_estimators=100, reg_alp

...

[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=2, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_depth=10, min_child_weight=1, n_estimators=200, reg_alpha=0.1, reg_lambda=2, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=2, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=2, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=2, subsample=1.0; total time=   0.1s


/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:46] WARNING: /w

[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=2, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=3, min_child_weight=1, n_estimators=300, reg_alpha=0.01, reg_lambda=2, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=3, min_child_weight=1, n_estimators=300, reg_alpha=0.01, reg_lambda=2, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=3, min_child_weight=1, n_estimators=300, reg_alpha=0.01, reg_lambda=2, subsample=0.6; total time=   0.1s


/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/workspaces/decision-tree---random-forest---boosting-algorithms-project/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:43:47] WARNING: /w

[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=3, min_child_weight=1, n_estimators=300, reg_alpha=0.01, reg_lambda=2, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=3, min_child_weight=1, n_estimators=300, reg_alpha=0.01, reg_lambda=2, subsample=0.6; total time=   0.1s
Best Hyperparameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.6}


In [19]:
# optimized model

best_xgb = random_search.best_estimator_
best_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [20]:
# optimized y-prediction

opt_y_pred = best_xgb.predict(X_test)
opt_y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [21]:
# Compare reports and accuracy score

from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, classification_report

opt_report = classification_report(y_test, opt_y_pred)
print(f'Optimized Accuracy Score: {accuracy_score(y_test, opt_y_pred)}\n')
print(opt_report)

report = classification_report(y_test, y_pred)
print(f'Regular Accuracy Score:\n {accuracy_score(y_test, y_pred)}')
print(report)

Optimized Accuracy Score: 0.7862068965517242

              precision    recall  f1-score   support

           0       0.80      0.93      0.86       102
           1       0.73      0.44      0.55        43

    accuracy                           0.79       145
   macro avg       0.76      0.69      0.71       145
weighted avg       0.78      0.79      0.77       145

Regular Accuracy Score:
 0.8344827586206897
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       102
           1       0.71      0.74      0.73        43

    accuracy                           0.83       145
   macro avg       0.80      0.81      0.80       145
weighted avg       0.84      0.83      0.84       145



In [ ]:
# # Save Model

# dump(model, open('../model/xgboost_model-diabetes_predictor.sav', 'wb'))